In [1]:
# Create custom gym environments from scratch — A stock market example
# https://towardsdatascience.com/creating-a-custom-openai-gym-environment-for-stock-trading-be532be3910e

# Rendering elegant stock trading agents using Matplotlib and Gym
# https://towardsdatascience.com/visualizing-stock-trading-agents-using-matplotlib-and-gym-584c992bc6d4

# Creating Bitcoin trading bots don’t lose money
# https://towardsdatascience.com/creating-bitcoin-trading-bots-that-dont-lose-money-2e7165fb0b29

# Optimizing deep learning trading bots using state-of-the-art techniques
# https://towardsdatascience.com/using-reinforcement-learning-to-trade-bitcoin-for-massive-profit-b69d0e8f583b

In [5]:
import random
import pandas as pd
import numpy as np

import gym
from gym import spaces

MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 20000

INITIAL_ACCOUNT_BALANCE = 10000

class StockTradingEnv(gym.Env):
    metadata = {'render.modes':['human']}
    
    def __init__(self, df ):
        super(StockTradingEnv, self).__init__()
        
        self.df = df
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)
        
        # 
        self.action_space = spaces.Box(
            low=np.array([0,0]), high=np.array([3,1]), dtype=np.float16)
        
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(6,6), dtype=np.float16)
        
    def step(self, action):
        self._take_action(action)
        
        self.current_step += 1
        
        if self.current_step > len(self.df.loc[:, 'Open'].values) -6:
            self.current_step = 0
        
        delay_modifier = (self.current_step / MAX_STEPS)
        
        reward = self.balance * delay_modifier
        done = self.net_worth <= 0
        
        obs = self._next_observation()
        
        return obs, reward, done, {}
    
    def _take_action(self, action):
        
        current_price = random.uniform(
            self.df.loc[self.current_step, "Open"],
            self.df.loc[self.current_step, "Close"])
        
        action_type = action[0]
        amount = action[1]
        
        if action_type < 1:
            # Buy amount % of balance in shares
            total_possible = int(self.balance / current_price)
            shares_bought = int(total_possible * amount)
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price
            
            self.balance -= additional_cost
            self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought
            
        elif action_type < 2:
            # Sell amount % of shares held
            shares_sold = int(self.shares_held * amount)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price
            
        self.net_worth = self.balance + self.shares_held * current_price
        
        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth
            
        if self.shares_held == 0:
            self.cost_basis = 0
            
        
    def reset(self):
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0
        
        self.current_step = random.randint(0, len(self.df.loc[:,"Open"].values) -6)
        
        return self._next_observation()
    
    def _next_observation(self):
        frame = np.array([
            self.df.loc[self.current_step: self.current_step +5, "Open"].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +5, 'High'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +5, 'Low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +5, 'Close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +5, 'Volume'].values / MAX_NUM_SHARES,
        ])
        
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)
        
        return obs
        

    def render(self, filename='render.txt'):
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE
                
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(f'Shares held: {self.shares_held} \n (Total sold: {self.total_shares_sold})')
        print(f'Avg cost for held shares: {self.cost_basis}\n(Total sales value: {self.total_sales_value})')
        print(f'Net worth: {self.net_worth}\n(Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')
        
        

In [6]:
import gym
import json
import datetime as dt

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

#from env.StockTradingEnv import StockTradingEnv

import pandas as pd

df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

env = DummyVecEnv([lambda: StockTradingEnv(df)])

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=20000)

obs = env.reset()

for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

/Users/alexanderwu/Code/RL_Trading/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars


---------------------------------------
| approxkl           | 9.489275e-08   |
| clipfrac           | 0.0            |
| explained_variance | -1.79e-06      |
| fps                | 270            |
| n_updates          | 1              |
| policy_entropy     | 2.838216       |
| policy_loss        | -5.5496115e-05 |
| serial_timesteps   | 128            |
| time_elapsed       | 3.1e-06        |
| total_timesteps    | 128            |
| value_loss         | 28232582.0     |
---------------------------------------
--------------------------------------
| approxkl           | 9.3384685e-08 |
| clipfrac           | 0.0           |
| explained_variance | -2.38e-07     |
| fps                | 407           |
| n_updates          | 2             |
| policy_entropy     | 2.8391924     |
| policy_loss        | -7.690047e-05 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.475         |
| total_timesteps    | 256           |
| value_loss         | 64103096.0    |
------------

--------------------------------------
| approxkl           | 0.00012500929 |
| clipfrac           | 0.0           |
| explained_variance | 0.00012       |
| fps                | 684           |
| n_updates          | 17            |
| policy_entropy     | 2.8475776     |
| policy_loss        | -0.0013280166 |
| serial_timesteps   | 2176          |
| time_elapsed       | 4.16          |
| total_timesteps    | 2176          |
| value_loss         | 319166.75     |
--------------------------------------
--------------------------------------
| approxkl           | 0.00032801006 |
| clipfrac           | 0.0           |
| explained_variance | 9.79e-05      |
| fps                | 672           |
| n_updates          | 18            |
| policy_entropy     | 2.8474889     |
| policy_loss        | -0.0037186234 |
| serial_timesteps   | 2304          |
| time_elapsed       | 4.35          |
| total_timesteps    | 2304          |
| value_loss         | 553041.7      |
-------------------------

--------------------------------------
| approxkl           | 0.0024099383  |
| clipfrac           | 0.0078125     |
| explained_variance | -1.43e-06     |
| fps                | 386           |
| n_updates          | 34            |
| policy_entropy     | 2.8486943     |
| policy_loss        | -0.0011757119 |
| serial_timesteps   | 4352          |
| time_elapsed       | 7.53          |
| total_timesteps    | 4352          |
| value_loss         | 651495300.0   |
--------------------------------------
--------------------------------------
| approxkl           | 0.00029484322 |
| clipfrac           | 0.0           |
| explained_variance | -2.62e-06     |
| fps                | 490           |
| n_updates          | 35            |
| policy_entropy     | 2.848849      |
| policy_loss        | -0.001139068  |
| serial_timesteps   | 4480          |
| time_elapsed       | 7.86          |
| total_timesteps    | 4480          |
| value_loss         | 888139460.0   |
-------------------------

--------------------------------------
| approxkl           | 0.0008735374  |
| clipfrac           | 0.0           |
| explained_variance | 0.000676      |
| fps                | 522           |
| n_updates          | 51            |
| policy_entropy     | 2.8394308     |
| policy_loss        | -0.0027797928 |
| serial_timesteps   | 6528          |
| time_elapsed       | 11.3          |
| total_timesteps    | 6528          |
| value_loss         | 356386.97     |
--------------------------------------
-------------------------------------
| approxkl           | 0.0002796581 |
| clipfrac           | 0.0          |
| explained_variance | 9.47e-05     |
| fps                | 527          |
| n_updates          | 52           |
| policy_entropy     | 2.837365     |
| policy_loss        | 0.002755682  |
| serial_timesteps   | 6656         |
| time_elapsed       | 11.5         |
| total_timesteps    | 6656         |
| value_loss         | 342013.25    |
-------------------------------------

...

Step: 4240
Balance: 5396.095631863494
Shares held: 10 
 (Total sold: 433)
Avg cost for held shares: 102.83677246646769
(Total sales value: 58652.73048974434)
Net worth: 6482.37649537037
(Max net worth: 11552.062081145852)
Profit: -3517.6235046296297
Step: 4241
Balance: 5396.095631863494
Shares held: 10 
 (Total sold: 433)
Avg cost for held shares: 102.83677246646769
(Total sales value: 58652.73048974434)
Net worth: 6486.057664502983
(Max net worth: 11552.062081145852)
Profit: -3513.9423354970168
Step: 4242
Balance: 5396.095631863494
Shares held: 10 
 (Total sold: 433)
Avg cost for held shares: 102.83677246646769
(Total sales value: 58652.73048974434)
Net worth: 6485.903852423771
(Max net worth: 11552.062081145852)
Profit: -3514.0961475762288
Step: 4243
Balance: 5396.095631863494
Shares held: 10 
 (Total sold: 433)
Avg cost for held shares: 102.83677246646769
(Total sales value: 58652.73048974434)
Net worth: 6484.993452759163
(Max net worth: 11552.062081145852)
Profit: -3515.00654724083

Step: 4306
Balance: 6526.232907255632
Shares held: 1 
 (Total sold: 472)
Avg cost for held shares: 115.61282024357547
(Total sales value: 63155.722934969795)
Net worth: 6652.602866771817
(Max net worth: 11552.062081145852)
Profit: -3347.3971332281826
Step: 4307
Balance: 6526.232907255632
Shares held: 1 
 (Total sold: 472)
Avg cost for held shares: 115.61282024357547
(Total sales value: 63155.722934969795)
Net worth: 6653.317305639649
(Max net worth: 11552.062081145852)
Profit: -3346.682694360351
Step: 4308
Balance: 6526.232907255632
Shares held: 1 
 (Total sold: 472)
Avg cost for held shares: 115.61282024357547
(Total sales value: 63155.722934969795)
Net worth: 6653.967839665011
(Max net worth: 11552.062081145852)
Profit: -3346.0321603349894
Step: 4309
Balance: 6526.232907255632
Shares held: 1 
 (Total sold: 472)
Avg cost for held shares: 115.61282024357547
(Total sales value: 63155.722934969795)
Net worth: 6654.643346395462
(Max net worth: 11552.062081145852)
Profit: -3345.35665360453

Step: 4375
Balance: 6794.49710600981
Shares held: 0 
 (Total sold: 524)
Avg cost for held shares: 0
(Total sales value: 69832.66648797225)
Net worth: 6794.49710600981
(Max net worth: 11552.062081145852)
Profit: -3205.5028939901904
Step: 4376
Balance: 6794.49710600981
Shares held: 0 
 (Total sold: 524)
Avg cost for held shares: 0
(Total sales value: 69832.66648797225)
Net worth: 6794.49710600981
(Max net worth: 11552.062081145852)
Profit: -3205.5028939901904
Step: 4377
Balance: 6794.49710600981
Shares held: 0 
 (Total sold: 524)
Avg cost for held shares: 0
(Total sales value: 69832.66648797225)
Net worth: 6794.49710600981
(Max net worth: 11552.062081145852)
Profit: -3205.5028939901904
Step: 4378
Balance: 6794.49710600981
Shares held: 0 
 (Total sold: 524)
Avg cost for held shares: 0
(Total sales value: 69832.66648797225)
Net worth: 6794.49710600981
(Max net worth: 11552.062081145852)
Profit: -3205.5028939901904
Step: 4379
Balance: 6794.49710600981
Shares held: 0 
 (Total sold: 524)
Avg 

Step: 4425
Balance: 292.91708531593576
Shares held: 52 
 (Total sold: 524)
Avg cost for held shares: 125.03038501334373
(Total sales value: 69832.66648797225)
Net worth: 6319.469932252433
(Max net worth: 11552.062081145852)
Profit: -3680.530067747567
Step: 4426
Balance: 292.91708531593576
Shares held: 52 
 (Total sold: 524)
Avg cost for held shares: 125.03038501334373
(Total sales value: 69832.66648797225)
Net worth: 6273.68405220475
(Max net worth: 11552.062081145852)
Profit: -3726.3159477952504
Step: 4427
Balance: 292.91708531593576
Shares held: 52 
 (Total sold: 524)
Avg cost for held shares: 125.03038501334373
(Total sales value: 69832.66648797225)
Net worth: 6305.232504623046
(Max net worth: 11552.062081145852)
Profit: -3694.767495376954
Step: 4428
Balance: 292.91708531593576
Shares held: 52 
 (Total sold: 524)
Avg cost for held shares: 125.03038501334373
(Total sales value: 69832.66648797225)
Net worth: 6295.06937896783
(Max net worth: 11552.062081145852)
Profit: -3704.9306210321

Step: 4483
Balance: 48.55775894132057
Shares held: 49 
 (Total sold: 627)
Avg cost for held shares: 116.33436629825712
(Total sales value: 81003.35224011619)
Net worth: 5828.346253931671
(Max net worth: 11552.062081145852)
Profit: -4171.653746068329
Step: 4484
Balance: 48.55775894132057
Shares held: 49 
 (Total sold: 627)
Avg cost for held shares: 116.33436629825712
(Total sales value: 81003.35224011619)
Net worth: 5676.896672287241
(Max net worth: 11552.062081145852)
Profit: -4323.103327712759
Step: 4485
Balance: 48.55775894132057
Shares held: 49 
 (Total sold: 627)
Avg cost for held shares: 116.33436629825712
(Total sales value: 81003.35224011619)
Net worth: 5881.164654758849
(Max net worth: 11552.062081145852)
Profit: -4118.835345241151
Step: 4486
Balance: 48.55775894132057
Shares held: 49 
 (Total sold: 627)
Avg cost for held shares: 116.33436629825712
(Total sales value: 81003.35224011619)
Net worth: 5938.722816899096
(Max net worth: 11552.062081145852)
Profit: -4061.277183100904


In [7]:
import IPython

In [8]:
import tensorflow as tf; print(tf.__version__)

1.13.2


In [5]:
!which jupyter

/Users/alexanderwu/Code/RL_Trading/bin/jupyter
